In [1]:
import netCDF4 as nc

In [218]:
wd = xr.open_dataset("../build/weather_data/ALB_1.nc")

In [228]:
print(wd.variables["longitude"][:])
print(wd.variables["latitude"][:])
print(len(wd.variables["time"][:]))

<xarray.IndexVariable 'longitude' (longitude: 1)>
array([20.], dtype=float32)
Attributes:
    units:      degrees_east
    long_name:  longitude
<xarray.IndexVariable 'latitude' (latitude: 1)>
array([41.125], dtype=float32)
Attributes:
    units:      degrees_north
    long_name:  latitude
52584


In [265]:
from pathlib import Path
import xarray as xr
import pandas
import numpy as np


def join_netcdfs(datasets: list[xr.Dataset]) -> xr.Dataset:
    r"""
    Joins a list of xarray datasets into a single dataset.
    """
    dataset_joined = xr.concat(datasets, dim="location")

    # dataset_joined = dataset_joined.rename({"concat_dim": "location"})

    #dataset_joined = dataset_joined.reset_coords("latitude", drop=True)
    return dataset_joined

paths_windspeed = Path("../build/weather_data/").glob("*.nc")
list_windspeed = []
for path in paths_windspeed:
    data = xr.open_dataset(path)
    data = data.assign_coords(location=path.stem)
    data = data.reset_index(["latitude", "longitude"]).reset_coords(["latitude", "longitude"]).squeeze()
    list_windspeed.append(data)

windspeeds = join_netcdfs(list_windspeed)


In [307]:
def calculate_capacity_factors(windspeed: xr.Dataset, power_curve: xr.Dataset) -> xr.Dataset:
    r"""
    Calculate capacity factors from wind speed data and a power curve.
    """
    data = windspeed.copy()

    # calculate absolute wind speed
    data = data.assign(
        windspeed = lambda x: np.sqrt(x.u**2 + x.v**2)  # pythagoras
    )

    # round to 1 decimal to be able to merge with power curve
    data = data.assign(
        windspeed = lambda x: np.round(x.windspeed, decimals=1)
    )

    # apply power curve
    capacity_factors = power_curve["capfac"].sel(windspeed=data.windspeed, method="nearest")

    return capacity_factors

# load power curve
power_curve = pd.read_csv("../data/power_curves/AWE_500kw_softwing.csv", index_col="windspeed")
power_curve = xr.Dataset.from_dataframe(power_curve)

# subset windspeeds
#windspeeds = windspeeds.sel(time=slice('2013-01-01', '2013-01-31'), location=["AUT_1", "DEU_1"])

# calculate capacity factors
capacity_factors = calculate_capacity_factors(windspeeds, power_curve)

# save
capacity_factors.to_dataframe().reset_index().pivot(index="time", columns="location", values=["windspeed", "capfac"])

windspeed                                                  \
location                ALB_1 AUT_1 AUT_2 AUT_3 BEL_1 BGR_1 BIH_1 CHE_1 CHE_2   
time                                                                            
2013-01-01 00:00:00       2.6   7.3   5.6   4.5  19.9   0.6   1.9   1.6   2.9   
2013-01-01 01:00:00       1.8   7.3   5.6   4.8  19.8   0.4   2.6   1.6   2.7   
2013-01-01 02:00:00       1.3   6.9   5.8   5.2  18.1   0.4   3.2   1.4   2.3   
2013-01-01 03:00:00       1.2   6.7   6.0   5.4  16.7   0.3   3.5   1.2   2.3   
2013-01-01 04:00:00       1.1   6.6   5.8   5.0  16.1   0.7   3.7   1.1   2.4   
...                       ...   ...   ...   ...   ...   ...   ...   ...   ...   
2018-12-31 19:00:00       8.1   5.3   6.9   8.6   6.5   2.4   6.6   1.9   7.8   
2018-12-31 20:00:00       8.7   5.3   6.8   8.3   7.2   2.3   6.6   2.0   7.7   
2018-12-31 21:00:00       9.3   5.1   6.5   7.9   7.6   2.3   6.2   1.6   7.6   
2018-12-31 22:00:00       8.8   5.0   6.9   8.0   8.5   2.1   5.8   1.5   7.9   
2018-12-31 23:00:00       9.3   4.5   6.7   7.5   9.3   1.8   6.1   1.3   6.9   

                           ... capfac                                      \
location            CYP_1  ...  BGR_1     BIH_1 CHE_1     CHE_2     CYP_1   
time                       ...                                              
2013-01-01 00:00:00   3.3  ...    0.0  0.000000   0.0  0.000000  0.000000   
2013-01-01 01:00:00   4.0  ...    0.0  0.000000   0.0  0.000000  0.000000   
2013-01-01 02:00:00   4.6  ...    0.0  0.000000   0.0  0.000000  0.007426   
2013-01-01 03:00:00   5.2  ...    0.0  0.000000   0.0  0.000000  0.037523   
2013-01-01 04:00:00   5.8  ...    0.0  0.000000   0.0  0.000000  0.102787   
...                   ...  ...    ...       ...   ...       ...       ...   
2018-12-31 19:00:00   5.2  ...    0.0  0.163827   0.0  0.329652  0.037523   
2018-12-31 20:00:00   5.6  ...    0.0  0.163827   0.0  0.310992  0.091480   
2018-12-31 21:00:00   5.9  ...    0.0  0.129766   0.0  0.293389  0.108954   
2018-12-31 22:00:00   6.5  ...    0.0  0.102787   0.0  0.349431  0.154554   
2018-12-31 23:00:00   6.7  ...    0.0  0.122421   0.0  0.195120  0.173656   

                                                                   
location                CZE_1     CZE_2 DEU_1     DEU_2     DEU_3  
time                                                               
2013-01-01 00:00:00  0.875140  1.000000   1.0  1.000000  1.000000  
2013-01-01 01:00:00  0.875140  1.000000   1.0  1.000000  1.000000  
2013-01-01 02:00:00  0.919781  1.000000   1.0  1.000000  1.000000  
2013-01-01 03:00:00  0.957128  1.000000   1.0  1.000000  1.000000  
2013-01-01 04:00:00  0.938269  1.000000   1.0  1.000000  1.000000  
...                       ...       ...   ...       ...       ...  
2018-12-31 19:00:00  0.065380  0.246335   1.0  0.957128  0.329652  
2018-12-31 20:00:00  0.129766  0.102787   1.0  1.000000  0.525415  
2018-12-31 21:00:00  0.163827  0.028427   1.0  1.000000  0.704318  
2018-12-31 22:00:00  0.154554  0.129766   1.0  1.000000  0.883891  
2018-12-31 23:00:00  0.195120  0.096969   1.0  1.000000  0.919781  

[52584 rows x 30 columns]